## Import Data

In [ ]:
# helper.py
from helper import *

# pytorch
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

# set up GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [21]:
# load data
df = pd.read_csv('./data/clean/features.csv')

# drop all aggregated cols
df = df.drop(columns=[col for col in df.columns if 'mean' in col or 'std' in col])

# check
df.shape, len(set(df.player))

((1099, 66), 237)

In [ ]:
def create_seq(df, seq_len):
    '''
    Pack data into sequences of seq_len NFL seasons. Return features and target variables.
    
    Parameters:
    df (pandas.DataFrame) - dataframe to create the sequences from
    seq_len (int) - number of seasons to use to predict the target
    
    Returns:
    X (numpy.ndarray) - features
    y (numpy.ndarray) - targets
    '''
    
    # get players with at least seq_len seasons
    df_grouped = df.groupby('player').filter(lambda x: len(x) >= seq_len)

    # init empty lists for sequences and labels
    sequences = []
    labels = []

    # non-feature cols
    non_feat_cols = ['player', 'team_name', 'year', 'target']

    # iterate over each player
    for player, group in df_grouped.groupby('player'):
        # iterate over the group to create sequences
        for i in range((len(group) - seq_len) + 1):
            # each sequence is a list of rows/features from two consecutive seasons
            sequence = group.iloc[i:(i + seq_len)].drop(non_feat_cols, axis=1).to_dict('records')
            
            # the label is the 'target' col from the final season in sequence
            label = group.iloc[i + seq_len - 1]['target']
            
            sequences.append(sequence)
            labels.append(label)

    # unpack the dicts in each sequence, fill null values with 0
    unpacked_sequences = [[[val for val in list(d.values())] for d in sequence] for sequence in sequences]   

    # convert sequences and labels into np array
    X = np.array(unpacked_sequences)
    y = np.array(labels)

    return X, y

In [10]:
for seq_len in [2, 3, 4, 5]:
    X, y = create_seq(seq_len=seq_len, df=df)
    print(f'Using sequences of {seq_len} seasons:')
    print(f'Shape of X = {X.shape}')
    print(f'Shape of y = {y.shape}')
    print()

Using sequences of 2 seasons:
Shape of X = (862, 2, 62)
Shape of y = (862,)

Using sequences of 3 seasons:
Shape of X = (674, 3, 62)
Shape of y = (674,)

Using sequences of 4 seasons:
Shape of X = (529, 4, 62)
Shape of y = (529,)

Using sequences of 5 seasons:
Shape of X = (411, 5, 62)
Shape of y = (411,)



In [33]:
def create_player_histories(df):
    """
    Returns:
      X_pad: FloatTensor, shape (n_players, max_seasons, n_features)
      y:      FloatTensor, shape (n_players,)
      mask:   BoolTensor, shape (n_players, max_seasons)
      players: list of player names, length n_players
    """

    # non-feature columns
    non_feat_cols = ['player', 'team_name', 'year', 'target']

    # init lists
    sequences, labels, players = [], [], []

    # iterate through each player
    for player, g in df.groupby('player'):
        # sort
        g = g.sort_values('year').reset_index(drop=True)
        
        # get length of group
        n = len(g)

        # cache the feature matrix once
        feat_mat = g.drop(columns=non_feat_cols).values

        # for each window end i = 0 ... n-1, make a sequence of [0:i] predicting target[i]
        for i in range(n):
            # seasons 0 through i
            seq = torch.tensor(feat_mat[: i+1, :], dtype=torch.float32)
            
            # target for season i
            lbl = torch.tensor(g['target'].iloc[i], dtype=torch.float32)

            # append to lists
            sequences.append(seq)
            labels.append(lbl)
            players.append(player)

    # pad to longest sequence
    X_pad = pad_sequence(sequences, batch_first=True)  

    # build mask so model knows which timesteps are real
    lengths = torch.tensor([seq.size(0) for seq in sequences])
    max_len = X_pad.size(1)
    mask = torch.arange(max_len)[None, :] < lengths[:, None]

    # create labels
    y = torch.stack(labels)

    return X_pad, y, mask, players

In [34]:
X_pad, y, mask, players = create_player_histories(df)

In [45]:
X_pad.shape

torch.Size([1099, 18, 62])

In [46]:
X_pad[0].shape

torch.Size([18, 62])

In [20]:
# StandardScaler won't accept 3-dimensional data as input, so we create a function
def standardize_3d(X_train, X_val=None):
    '''
    
    Reshapes X array from 3D to 2D and standardizes.
    
    Parameters
    X_train (np.array) - X_train array
    X_val: (np.array) - X_val array
    
    Returns
    X_train_standardized_3d (np.array) - standardized X_train array in 3-dimensional form
    X_val_standardized_3d (np.array) - standardized X_val array in 3-dimensional form
    
    '''
    
    if isinstance(X_val, np.ndarray):
        # reshape the data to be 2D -> (n_samples * n_time_steps, n_features)
        X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
        X_val_reshaped = X_val.reshape(-1, X_val.shape[-1])

        # create scaler, fit on the training data
        scaler = StandardScaler()
        scaler.fit(X_train_reshaped)

        # transform the data
        X_train_standardized_2d = scaler.transform(X_train_reshaped)
        X_val_standardized_2d = scaler.transform(X_val_reshaped)

        # reshape back to 3D
        X_train_standardized_3d = X_train_standardized_2d.reshape(X_train.shape)
        X_val_standardized_3d = X_val_standardized_2d.reshape(X_val.shape)

        return X_train_standardized_3d, X_val_standardized_3d
    
    else:
        # reshape the data to be 2D -> (n_samples * n_time_steps, n_features)
        X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])

        # create scaler, fit on the training data
        scaler = StandardScaler()
        scaler.fit(X_train_reshaped)

        # transform the data
        X_train_standardized_2d = scaler.transform(X_train_reshaped)

        # reshape back to 3D
        X_train_standardized_3d = X_train_standardized_2d.reshape(X_train.shape)

        return X_train_standardized_3d

In [280]:
def create_loaders(X, y, batch_size, test_size, random_state=random_state):
    '''
    
    
    Split data into train and val sets, standardize data, return training and valing dataloaders.
    
    Parameters:
    X (numpy.ndarray) - features
    y (numpy.ndarray) - targets
    batch_size (int) - batch size to be used in the training dataloader
    test_size (float) - percentage of data to be set aside as validation set
    
    
    Returns:
    train_loader (torch.utils.data.DataLoader) - train dataloader
    val_loader (torch.utils.data.DataLoader) - val dataloader 
    
    
    
    '''
    
    
    
    # typecast for compatibility
    y = y.astype('float32')
    
    if test_size > 0:
        # train/val split
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=random_state)
        
        # standardize the data
        X_train_standardized, X_val_standardized = standardize_3d(X_train, X_val)

        # convert sequences and labels into tensors
        X_train_tensor = torch.tensor(X_train_standardized, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val_standardized, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)
        y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(-1)

        # datasets
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

        # dataloaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=X_val_tensor.shape[0], shuffle=False)

        return train_loader, val_loader
    
    # single dataloaders for final generalization stage
    else:
        # standardize the data
        X_train_standardized = standardize_3d(X)

        # convert sequences and labels into tensors
        X_train_tensor = torch.tensor(X_train_standardized, dtype=torch.float32)
        y_train_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

        # datasets
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

        # dataloaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

        return train_loader

In [58]:
def train_val(model, optimizer, train_loader, val_loader, patience=15, device=device):
    
    # loss function
    criterion = nn.MSELoss()
    
    # 500 epochs
    num_epochs = 500

    # early stopping
    stopped = False
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    # training loop
    for epoch in range(num_epochs):
        # training mode
        model.train()

        # sum of squared error, preds, and y. reset at the start of every epoch
        sse = 0
        epoch_preds = []
        epoch_y = []

        # train batches
        for x, y in train_loader:
            # put x and y on gpu
            x = x.to(device)
            y = y.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward pass
            train_preds = model(x)
            # calc loss
            train_loss = criterion(train_preds, y)
            # backward pass
            train_loss.backward()
            # optimize
            optimizer.step()

            # accumulate squared errors
            sse += ((train_preds - y) ** 2).sum().item()

            # get preds and y from batch to calc r2
            epoch_preds.extend(train_preds.numpy(force=True))
            epoch_y.extend(y.numpy(force=True))

        # train metrics
        train_rmse = np.sqrt(sse / len(train_loader.dataset))
        train_r2 = r2_score(epoch_y, epoch_preds)



        # validation
        model.eval()

        with torch.inference_mode():
            # val loop (single batch)
            for x, y in val_loader:
                # put x and y on gpu
                x = x.to(device)
                y = y.to(device)
                
                # forward pass
                val_preds = model(x)
                # calc loss
                val_loss = criterion(val_preds, y)

        # val metrics
        val_rmse = np.sqrt(((val_preds - y) ** 2).sum().item() / y.shape[0])
        val_r2 = r2_score(y.numpy(force=True), val_preds.numpy(force=True))

    
    
        # early stopping
        if val_rmse < best_val_loss:
            best_val_loss = val_rmse
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

            # break out of train loop if we reach patience value
            if epochs_without_improvement == patience:
                stopped = True
                stopped_epoch = epoch + 1
                print(f'Early stopping on Epoch {stopped_epoch}.')
                break



    # get number of epochs that the model was trained for            
    if stopped:
        num_epochs = stopped_epoch



    return train_rmse, train_r2, val_rmse, val_r2, num_epochs, patience

In [59]:
# recurrent neural network
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=1, dropout=0):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True, num_layers=num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out
    
    
    
# long-short-term-memory rnn
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=1, dropout=0):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, num_layers=num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out
    
    
    
# gated-recurrent-unit rnn
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=1, dropout=0):
        super(GRU, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True, num_layers=num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

# Bayesian Optimization
- Iterating through parameter space to find best sequence model
- I want to find the best model for each specific sequence length. I will perform a round of Bayesian optimization for sequences of 2, 3, and 4 seasons.

In [66]:
# all features
all_feats = df.drop(['player', 'team_name', 'year', 'target'], axis=1).columns.tolist()

In [67]:
def objective_function(hidden_dim, num_layers, dropout, batch_size):
    # cast continuous values to int
    hidden_dim = int(hidden_dim)
    num_layers = int(num_layers)
    batch_size = int(batch_size)
    
    # create sequences
    X, y = create_seq(feature_subset=all_feats, seq_len=2, df=df)
    
    # create dataloaders
    train_loader, val_loader = create_loaders(X, y, test_size=0.1, batch_size=batch_size)

    # create model
    model = RNN(input_dim=66, hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout).to(device)
        
    # create optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    
    # train model
    train_rmse, train_r2, val_rmse, val_r2, num_epochs, patience = train_val(model, optimizer, train_loader, val_loader)
    
    # return the negative of the validation metric (since Bayesian optimization minimizes)
    return -val_rmse

In [68]:
# define bounds of each hyperparameter
pbounds = {
    'hidden_dim': (4, 1025),
    'num_layers': (2, 17),
    'dropout': (0, 0.9),
    'batch_size': (1, 129)
}

# create the Bayesian optimizer
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    random_state=random_state,
    allow_duplicate_points=True
)

In [69]:
# iterate through feature space
optimizer.maximize(init_points=10, n_iter=100)

|   iter    |  target   | batch_... |  dropout  | hidden... | num_la... |
-------------------------------------------------------------------------
Early stopping on Epoch 20.
| 1         | -13.31    | 2.328     | 0.4517    | 510.2     | 4.007     |
Early stopping on Epoch 93.
| 2         | -12.43    | 19.19     | 0.1967    | 431.3     | 5.722     |
Early stopping on Epoch 54.
| 3         | -13.34    | 11.76     | 0.3109    | 174.3     | 15.18     |
Early stopping on Epoch 21.
| 4         | -72.67    | 122.7     | 0.03487   | 717.8     | 10.59     |
Early stopping on Epoch 73.
| 5         | -13.32    | 115.9     | 0.6002    | 563.3     | 12.54     |
Early stopping on Epoch 39.
| 6         | -14.19    | 50.47     | 0.625     | 846.2     | 8.985     |
Early stopping on Epoch 151.
| 7         | -12.06    | 121.4     | 0.7234    | 1.008e+03 | 4.545     |
Early stopping on Epoch 28.
| 8         | -13.3     | 67.7      | 0.851     | 667.2     | 14.91     |
Early stopping on Epoch 88.
| 9    

Early stopping on Epoch 208.
| 56        | -13.33    | 90.87     | 0.2738    | 134.4     | 16.37     |
Early stopping on Epoch 63.
| 57        | -12.27    | 26.34     | 0.9       | 209.5     | 2.0       |
Early stopping on Epoch 32.
| 58        | -13.28    | 41.07     | 0.9       | 749.1     | 17.0      |
Early stopping on Epoch 75.
| 59        | -12.38    | 76.71     | 0.9       | 581.6     | 2.0       |
Early stopping on Epoch 469.
| 60        | -13.34    | 128.5     | 0.8691    | 60.56     | 13.48     |
Early stopping on Epoch 42.
| 61        | -12.95    | 1.0       | 0.0       | 346.2     | 2.0       |
Early stopping on Epoch 16.
| 62        | -67.17    | 88.12     | 0.0       | 851.2     | 17.0      |
Early stopping on Epoch 62.
| 63        | -12.23    | 92.22     | 0.9       | 791.9     | 2.0       |
Early stopping on Epoch 354.
| 64        | -13.33    | 88.78     | 0.9       | 63.16     | 17.0      |
Early stopping on Epoch 216.
| 65        | -13.33    | 70.41     | 0.4259    | 

In [70]:
# look at params that gave lowest validation RMSE
best_hyperparams = optimizer.max['params']
best_hyperparams

{'batch_size': 32.03638856422199,
 'dropout': 0.9,
 'hidden_dim': 100.74795411866992,
 'num_layers': 2.0}

- Best val_rmse = __11.87__ with a sequence length of 2.

# 2023 predictions
- This model will be trained on the entire train/val data, and then will predict 2023 offensive grade from the 2022 holdout set.
- Since we are using a sequence length of 2 seasons to predict the third, we will use 2021 & 2022 seasons (to predict 2022 target) as the test set.
- This means that players with under 3 seasons played can't be predicted on. I will use the best model (Random Forest) from [this notebook](./models_1.ipynb) to make these predictions.

In [97]:
# get the names of the 48 players that have 2023 targets
player_names_2023 = players_2022.player.values

# master_df includes 2022 rows. create a subset for these players, get players with at least 2 seasons
players_subset = master_df[master_df['player'].isin(player_names_2023)]
players_subset = players_subset.groupby('player').filter(lambda x: len(x) >= 2)

# get last two rows for each player
seq_test = players_subset.groupby('player').apply(lambda x: x.tail(2)).reset_index(drop=True)

In [98]:
# train sequences
X_train, y_train = create_seq(feature_subset=all_feats, seq_len=2, df=df)

# test sequences
X_test, y_test = create_seq(feature_subset=all_feats, seq_len=2, df=seq_test)

In [99]:
# 752 total examples to train on, 42 QBs to predict on
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((752, 2, 66), (752,), (42, 2, 66), (42,))

In [100]:
# create dataloaders
train_loader = create_loaders(X_train, y_train, test_size=0, batch_size=32)
test_loader = create_loaders(X_test, y_test, test_size=0, batch_size=len(X_test))

In [169]:
# best sequence model
best_seq = RNN(input_dim=len(all_feats), hidden_dim=98, num_layers=2, dropout=0.9).to(device)

# create optimizer
optimizer = torch.optim.AdamW(best_seq.parameters(), lr=0.001)

In [170]:
# loss function
criterion = nn.MSELoss()

# 100 epochs
num_epochs = 100

# training mode
best_seq.train()

# training loop
for epoch in range(num_epochs):  
    # train batches
    for x, y in train_loader:
        # put x and y on gpu
        x = x.to(device)
        y = y.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward pass
        train_preds = best_seq(x)
        # calc loss
        train_loss = criterion(train_preds, y)
        # backward pass
        train_loss.backward()
        # optimize
        optimizer.step()

In [171]:
# test set
best_seq.eval()

with torch.inference_mode():
    for x, y in test_loader:
        # put x and y on gpu
        x = x.to(device)
        y = y.to(device)

        # forward pass
        test_preds = best_seq(x)
        # calc loss
        test_loss = criterion(test_preds, y)

        # performance metrics
        rmse = np.sqrt(((test_preds - y) ** 2).sum().item() / y.shape[0])
        r2 = r2_score(y.numpy(force=True), test_preds.numpy(force=True))

print(f'RMSE: {rmse:.3f}')
print(f'R^2: {r2:.3f}')

RMSE: 12.895
R^2: 0.317


- On the 42/48 QBs who have 3+ seasons played, our model predicts their 2023 offensive grade with an RMSE of 12.895.

In [172]:
# get 2022 players who can't be predicted on with sequence model (players with less than 2 seasons)
players_subset = master_df[master_df['player'].isin(player_names_2023)]
players_subset = players_subset.groupby('player').filter(lambda x: len(x) < 2)

# get last row for each player
non_seq_test = players_subset.groupby('player').apply(lambda x: x.tail(1)).reset_index(drop=True)
non_seq_test.shape

(6, 70)

- 6 QBs in 2023 with less than 3 seasons played.

In [173]:
# best random forest
best_rf = RandomForestRegressor(random_state=random_state, min_samples_split=112)

# features and target
X_train = df[all_feats]
y_train = df.target
X_test = non_seq_test[all_feats]
y_test = non_seq_test.target

# create pieline
pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('rf', best_rf)
    ])

# train on entire dataset
pipeline.fit(X_train, y_train)

# predict
preds = pipeline.predict(X_test)

rmse = mean_squared_error(y_test, preds, squared=False)
r2 = r2_score(y_test, preds)

print(f'RMSE: {rmse:.3f}')
print(f'R^2: {r2:.3f}')

RMSE: 11.658
R^2: 0.245


In [194]:
# combine preds from the two models
y_pred = np.concatenate([test_preds.squeeze(-1).cpu().numpy(), preds])

# get true values
y_true = np.concatenate([y.squeeze(-1).cpu().numpy(), y_test])

# look at overall performance
rmse = mean_squared_error(y_true, y_pred, squared=False)
r2 = r2_score(y_true, y_pred)
print(f'RMSE: {rmse:.3f}')
print(f'R^2: {r2:.3f}')

RMSE: 12.747
R^2: 0.338


- Using an RNN with a sequence length of 2 (paired with best Random Forest), we achieve a RMSE of __12.75__.
- This is worse performance than just the Random Forest from [models_1](./models_1.ipynb).

In [291]:
# player names
player_names = seq_test.player.unique().tolist() + non_seq_test.player.unique().tolist()

# teams
team_names = []
for _, group in seq_test.groupby('player'):
    team_names.append(group.iloc[-1].team_name)

team_names.extend(non_seq_test.team_name.tolist())